# Diagnostics of mock photometry

_Alex Malz (GCCL@RUB)_

In [ ]:
import corner
from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import manifold

In [ ]:
toppath = '/media/CRP6/Cosmology/recidivator/'

## start with examining GAMA data

In [ ]:
gamapath = toppath+'GAMA/SpecObjPhot.csv'

In [ ]:
data = pd.read_csv(gamapath)

In [ ]:
data.columns

### divide by field

In [ ]:
xlim = {}
xlim['G02'] = (30.2, 38.8)
xlim['G09'] = (129.0, 141.0)
xlim['G12'] = (174.0, 186.0)
xlim['G15'] = (211.5, 223.5)

ylim = {}
ylim['G02'] = (-10.5, -3.72)
ylim['G09'] = (-2., 3.)
ylim['G12'] = (-3., 2.)
ylim['G15'] = (-2., 3.)

### isolate photometry and redshift

In [ ]:
gama_fields ={}
for key in xlim.keys():
    gama_fields[key] = data.loc[(data['RA'] > xlim[key][0]) & (data['RA'] < xlim[key][1]) & (data['DEC'] > ylim[key][0]) & (data['DEC'] < ylim[key][1])][['Z']+['lsst'+band for band in 'grizy']]
#     gama_fields[key].to_csv(toppath+'GAMA/'+key+'SpecObjPhot.csv')

In [ ]:
corner.corner(gama_fields['G15'], xlim=(8, 24), ylim=(8, 24))

In [ ]:
for i in gama_fields['G15'].columns:
    plt.hist(gama_fields['G15'][i], bins=100)
    plt.xlabel(i)
    plt.show()

this is where I should filter them by which of 'grizy' they have (not counting 0s and NaNs) and color by those later

In [ ]:
for i in range(len('grizy')):
    print(list(combinations('grizy', i+1)))

want a better visualization for high-D data

In [ ]:
X = gama_fields['G15'].values

In [ ]:
tsne = manifold.TSNE()
res = tsne.fit_transform(X)

In [ ]:
plt.scatter(res[:, 0], res[:, 1], s=1, alpha=0.5)
plt.show()

## after environment classification

let's see if these problems are solved by looking at per-environment-class data, since that's what we'll be comparing regardless.

## next steps

* mag vs. redshift 
* color vs. redshift
* color vs. color

propensity scores (TODO: hunt down old code from stats class 2017)